In [29]:
import pandas
import numpy as np
from pprint import pprint
import operator

In [30]:
def replace_missing_values(df):
    #replace missing values in categorical/binary variables with mode
    #replace missing values in non-categorical variables with median

    #column indexes for categorical variables
    cat_indexes=[3,5,6,23,24,25,26,27,28,29,30,31,32,33]
    #column indexes for binary variables
    bin_indexes=[7,17,18,19,53,54,55,56,57]
    #column indexes for non-categorical variables
    nor_indexes=[2,4,15,16,20,21,22,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52]
    
    for index in range (2,59):
        name = list(df.iloc[:,[index]])[0]
        if index in nor_indexes:
            df[name] = df[name].fillna(df[name].median(axis=0))
        else:
            df[name] = df[name].fillna(df[name].mode()[0])
    return df


# just some evaluation if needed
from sklearn.metrics import accuracy_score, precision_score, recall_score

def evaluate(labels_test, pred):
    acc = accuracy_score(labels_test, pred)
    ps = precision_score(labels_test, pred)
    rs = recall_score(labels_test, pred)

    print "acc:", acc
    print "precision_score:", ps
    print "recall_score:", rs

In [31]:
df = pandas.read_csv('train.csv')
#convert -1 value to NaN
df.replace(to_replace=[-1],value=[np.nan], inplace = True)
if df.isnull().sum().sum() != 0:
    df = replace_missing_values(df)

In [32]:
df_test = pandas.read_csv('test.csv')
df_test.replace(to_replace=[-1],value=[np.nan], inplace = True)
idx = 1
new_col = np.zeros(df_test.shape[0]) 
df_test.insert(loc=idx, column='target', value=new_col)
if df_test.isnull().sum().sum() != 0:
    #if there are NaN values in the data frame, proceed to replace the NaN values with median/mode
    df_test = replace_missing_values(df_test)

In [33]:
X = df.iloc[:,2:59]
y = df.iloc[:,1]

In [34]:
# from sklearn.model_selection import train_test_split

# features_train, features_test, labels_train, labels_test = \
#     train_test_split(X, y, test_size=0.33, random_state=42)

In [51]:
from sklearn.feature_selection import SelectKBest, f_classif

num_of_features = 10

selector = SelectKBest(f_classif, k=num_of_features)
selector.fit(X, y)

new_X = selector.transform(X)

In [52]:
feature_list = list(X)

scores = selector.scores_

feature_select_idx_list = sorted(range(len(scores)), key=lambda i: scores[i])[-num_of_features:]

# To view the features being selected. The importance is sorted in ascending.
for i in feature_select_idx_list:
    print feature_list[i]

ps_reg_03
ps_ind_06_bin
ps_ind_07_bin
ps_reg_02
ps_ind_17_bin
ps_car_12
ps_car_13


In [53]:
# feature_score_dict = {}
# for i in range(len(feature_list)):
#     feature_score_dict[feature_list[i]] = scores[i]

# feature_score_tup_sorted = sorted(feature_score_dict.items(), key=operator.itemgetter(1), reverse=True)
# pprint (feature_score_tup_sorted[:10])

# col_names = list(df)
# col_names.pop(0)
# col_names.pop(0)

# print col_names
# print len(col_names)

In [54]:
# features_train, features_test, labels_train, labels_test = \
#     train_test_split(new_X, y, test_size=0.2, random_state=42)

In [8]:
from sklearn import tree
from sklearn.model_selection import GridSearchCV

params = {'min_samples_split':[45,50,55,60,65], 'criterion':('gini', 'entropy')}
clf_tree = tree.DecisionTreeClassifier()
clf = GridSearchCV(clf_tree, params)

clf.fit(new_X, y)

print "parameters:", clf.best_params_

#pred = clf.predict(features_test)

parameters: {'min_samples_split': 65, 'criterion': 'gini'}


In [55]:
from sklearn.naive_bayes import GaussianNB

clf = GaussianNB()

clf.fit(new_X, y)

GaussianNB(priors=None)

In [56]:
mod_feature_select_idx_list = []
for idx in feature_select_idx_list:
    mod_feature_select_idx_list.append(idx+2)
    
X_test = df_test.iloc[:, mod_feature_select_idx_list]

In [57]:
prob = clf.predict_proba(X_test)

y_result = pandas.DataFrame(prob)

#print y_result.head()
#evaluate(labels_test, pred)
#print "\n"

In [58]:
result_df = pandas.concat([df_test['id'], y_result[1]], axis=1, keys=['id', 'target'])

result_df.to_csv("result_df.csv", index=False)